In [ ]:
import numpy as np
from astropy.io import fits
import pandas as pd
from glob import glob
import matplotlib.pyplot as plt
from astropy.visualization import ZScaleInterval
import ipywidgets as widgets
from ipywidgets import interact

In [ ]:
cigale = "./Cigale/out-2nd/"

hdul = fits.open("./mastDownload/JWST/cigale-data.fits")
results = fits.open(cigale+"results.fits")

print(results[1].data.columns)

In [ ]:
for file in glob(cigale+"*_best_model.fits"):
        name = file.split("/")[-1].replace("_best_model.fits","")
        print(name)
        path = hdul[1].data[hdul[1].data["id"] == name]["spectrum"][0][1:]
        
        z = results[1].data[results[1].data["id"] == name]["bayes.universe.redshift"][0]
        dz = results[1].data[results[1].data["id"] == name]["bayes.universe.redshift_err"][0]
        chi = results[1].data[results[1].data["id"] == name]["best.reduced_chi_square"][0]
        
        with fits.open(file) as sed:
            SED_wave = sed[1].data["wavelength"]/1000
            SED_Fnu = sed[1].data["Fnu"]
            
        with fits.open(path) as data:
            Raw_wave = data[1].data["WAVELENGTH"]
            Raw_Fnu = data[1].data["FLUX"]*1e3
            mask = SED_wave > Raw_wave.min()
            mask = np.logical_and(mask, SED_wave < Raw_wave.max())
        
        SED_wave = SED_wave[mask]
        SED_Fnu = SED_Fnu[mask]
        
        plt.figure(figsize=(12,5))
        plt.subplot(2,1,1)
        plt.title(fr"z = {round(z,2)} +/- {round(dz,2)}, $\chi^2$ = {round(chi,2)}")
        plt.plot(SED_wave,SED_Fnu)
        plt.plot(Raw_wave,Raw_Fnu,linestyle='dashed')
        #plt.xscale("log")
        #plt.yscale("log")
        plt.grid()
        plt.xlabel(r"$\lambda$ (µm)")
        plt
        plt.subplot(2,1,2)

        path = path.replace("prism1",'prism')
        with fits.open(path.replace("x1d","s2d")) as s2d:
            z1, z2 = ZScaleInterval().get_limits(s2d[1].data)
            plt.imshow(s2d[1].data,origin="lower",vmin=z1,vmax=z2)
        plt.show()
        

In [ ]:
# Function to read and process spectra files
def read_spectra(file_path):
    data = pd.read_csv(file_path, sep=' ', skipfooter=1, skiprows=1, skipinitialspace=True, engine='python')
    data.columns = ["Intensity", "Wavelength", "Type", "Ref"]
    data["Wavelength"] /= 1e4
    return data

# Reading spectra data
He = read_spectra("./Test/He_spectra.txt")
H = read_spectra("./Test/H_spectra.txt")
O = read_spectra("./Test/O_spectra.txt")
C = read_spectra("./Test/C_spectra.txt")
N = read_spectra("./Test/N_spectra.txt")

HeI = He[He["Type"] == "HeI"]
HeII = He[He["Type"] == "HeII"]
OI = O[O["Type"] == "OI"]
OII = O[O["Type"] == "OII"]
CI = C[C["Type"] == "CI"]
CII = C[C["Type"] == "CII"]
NI = N[N["Type"] == "NI"]
NII = N[N["Type"] == "NII"]


# Function to update the plot based on redshift z
def update_plot(z):
    plt.figure(figsize=(12, 6))
    
    # Apply redshift transformation
    HeI_shifted = HeI["Wavelength"] * (1 + z)
    HeII_shifted = HeII["Wavelength"] * (1 + z)
    H_shifted = H["Wavelength"] * (1 + z)
    OI_shifted = OI["Wavelength"] * (1 + z)
    OII_shifted = OII["Wavelength"] * (1 + z)
    CI_shifted = CI["Wavelength"] * (1 + z)
    CII_shifted = CII["Wavelength"] * (1 + z)
    NI_shifted = NI["Wavelength"] * (1 + z)
    NII_shifted = NII["Wavelength"] * (1 + z)
    
    plt.bar(HeI_shifted, HeI["Intensity"], label="He I",width = (z+1)/1000)
    plt.bar(HeII_shifted, HeII["Intensity"], label="He II",width = (z+1)/1000)
    plt.bar(H_shifted, H["Intensity"], label="H I",width = (z+1)/1000)
    #plt.bar(OI_shifted, OI["Intensity"], label="O I",width = (z+1)/1000)
    #plt.bar(OII_shifted, OII["Intensity"], label="O II",width = (z+1)/1000)
    #plt.bar(CI_shifted, CI["Intensity"], label="C I",width = (z+1)/1000)
    #plt.bar(CII_shifted, CII["Intensity"], label="C II",width = (z+1)/1000)
    #plt.bar(NI_shifted, NI["Intensity"], label="N I",width = (z+1)/1000)
    #plt.bar(NII_shifted, NII["Intensity"], label="N II",width = (z+1)/1000)
    
    plt.vlines(91.2/1e3 * (1 + z), 0, 1000, color="blue")  # Adjust the upper limit as needed
    plt.legend()
    plt.xlabel("Wavelength (µm)")
    plt.ylabel("Intensity")
    plt.title(f"Spectra with Redshift z = {z}")
    plt.yscale('log')
    plt.xlim(0.6,5.3)
    plt.grid()
    plt.show()

# Create an interactive slider for redshift z
interact(update_plot, z=widgets.FloatSlider(value=0, min=0, max=16, step=0.05, description='Redshift z'));

In [ ]:
plt.figure()
plt.hist(results[1].data["best.reduced_chi_square"],bins=100)
plt.show()

plt.figure()
plt.hist(results[1].data["bayes.universe.redshift"],bins=50)
plt.show()

plt.figure()
plt.scatter(results[1].data["bayes.universe.redshift"],results[1].data["bayes.stellar.metallicity"])
#plt.yscale("log")
plt.show()

In [ ]:
for i,columns in enumerate(results[1].data.columns):
    if i == 0:
        continue
    try:
        unit = columns.unit
    except:
        unit = ""
    
    plt.figure()
    plt.scatter(results[1].data["bayes.universe.redshift"],results[1].data[columns.name])
    plt.grid()
    plt.xlabel("z")
    plt.ylabel(f"{columns.name} ({unit})")
    plt.yscale("log")
    plt.show()

In [ ]:
cigale = ["./Cigale/out-1st/results.fits","./Cigale/out-2nd/results.fits","./Cigale/out-3rd/results.fits","./Cigale/out-4th/results.fits"]
plt.figure(figsize=(24,6))
for _ in cigale:
    results = fits.open(_)
    plt.scatter(np.arange(0,616,1),results[1].data["best.reduced_chi_square"],label=_)
    
plt.ylabel(r"\chi^2")
plt.legend()
plt.show()

plt.figure(figsize=(24,6))
for _ in cigale:
    results = fits.open(_)
    plt.errorbar(np.arange(0,616,1),results[1].data["bayes.universe.redshift"],yerr=results[1].data["bayes.universe.redshift_err"],
                 linestyle='none',label=_,marker="o")
    
plt.grid()
plt.ylabel("z")
plt.legend()
plt.show()

In [ ]:
plt.figure()
for _ in cigale:
    results = fits.open(_)
    chi = results[1].data["best.reduced_chi_square"]
    plt.hist(chi,bins=50,label=_,alpha=0.5)
    print(len(chi),len(chi[~np.isnan(chi)]))
plt.legend()
plt.show()

In [ ]:
for _ in cigale:
    results = fits.open(_)
    z = results[1].data["bayes.universe.redshift"]
    dz = results[1].data["bayes.universe.redshift_err"]
    Z = results[1].data["bayes.stellar.metallicity"]
    dZ = results[1].data["bayes.stellar.metallicity_err"]
    plt.figure()
    plt.errorbar(z,Z,yerr=dZ,xerr=dz,linestyle="none",color='k')
    plt.xlabel("z")
    plt.ylabel("metallicity")
    plt.show()

In [ ]:
with fits.open(cigale[-1]) as results:
    table = results[1].data
    print(table[np.isnan(table["bayes.universe.redshift"])]["id"])